# [실습 3] T5 모델을 이용한 뉴스 요약 프로젝트

## 실습 목표

---

1. Hugging Face 프레임워크의 기능을 알아보고, 기초적인 사용법을 익힙니다.

2. T5 모델을 이용하여 뉴스 내용을 요약해보고, 이를 헤드라인과 비교해보는 실습을 진행합니다.

3. Rouge 평가지표를 이용하여 모델의 성능을 확인해봅니다.

# Hugging Face 프레임워크 기초 실습

이번 실습 시간에는 Hugging Face의 다양한 라이브러리와 API를 사용하는 방법을 익혀보겠습니다.  

![](https://huggingface.co/datasets/huggingface/brand-assets/resolve/main/hf-logo-with-title.svg)



Hugging Face는 자연어 처리(NLP) 분야에서 가장 인기 있는 딥러닝 모델 및 도구를 제공하는 플랫폼입니다.  

주로 `transformers` 라이브러리를 통해 다양한 사전 훈련된 모델을 제공하며, 연구자와 개발자들 사이에서 널리 사용됩니다.

저번 시간에 구현해본 BERT 모델이 기억나시나요? 모델을 구현하는 데에 꽤 많은 시간과 노력이 소요되었지만, `transformers`라이브러리를 이용한다면 아래처럼 쉽게 모델을 사용할 수 있습니다.

In [1]:
# transformers 라이브러리에서 문장 분류용 BERT 모델과 토크나이저 불러오기
import torch
from transformers import BertTokenizer, BertForSequenceClassification

#토크나이저
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

#모델
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
# 모델 정의

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


사용하려는 모델에 맞게 토크나이저도 제공하므로, 텍스트에 손쉽게 적용할 수 있습니다.

In [2]:
text = "안녕하세요, transformers를 사용합니다!" # '안녕하세요, transformers를 사용합니다! ' 문자를 넣어 줍니다
encoded_input = tokenizer(text, return_tensors='pt')    # 토크나이저에 텍스트를 입력하고, 모델을 가동할 프레임워크(pt = Pytorch)를 인자로 제공
encoded_input

{'input_ids': tensor([[   101,   9521, 118741,  35506,  24982,  48549,    117,  99662,  10901,
          11513,   9405,  24974,  33188,  48345,    106,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

토큰화된 텍스트는 다음과 같이 사전학습된 모델을 이용하여 추론에 사용됩니다.

In [3]:
with torch.no_grad():
    output = model(**encoded_input)
logits = output.logits

logits

tensor([[-0.0045,  0.0915]])

모델의 출력값은 Softmax 함수를 통해 확률로 표현할 수 있습니다.

In [4]:
probabilities = torch.softmax(logits, dim=-1)
probabilities

tensor([[0.4760, 0.5240]])

지금부터 간단한 예제들을 통해 텍스트 전처리, 토크나이저 사용 및 모델 사용법을 알려드리도록 하겠습니다.

## 1. 토크나이저

토크나이저를 사용하는 주요 이유는 자연어 처리 작업에서 텍스트 데이터를 모델이 이해할 수 있는 형식으로 변환하기 위함입니다.

### 1.1 공백 기반 토크나이저

가장 기본적인 형태의 토크나이저는 공백을 기반으로 단어를 나누어 토큰을 생성합니다. Python의 문자열 함수 중 `split()`을 통해 아래와 같이 진행합니다.

In [5]:
tokenized_text = text.split()
tokenized_text

['안녕하세요,', 'transformers를', '사용합니다!']

그러나 공백 단위로 단어를 나누게 되면 완벽하게 나뉘지 않을 뿐더러, 한국어 등 교착어에서는 더욱이나 성능이 떨어지게 됩니다.  

고로 대부분의 대규모 언어모델에서는 하위 단어 토큰화(Subword Tokenization) 등 단어의 의미를 더 잘게 나눠 해석할 수 있는 토크나이저를 선호합니다.  

모든 모델마다 저마다 최적의 효율을 내는 토크나이저가 따로 있으며, `transformers`라이브러리에서는 사전학습된 토크나이저를 손쉽게 불러올 수 있도록 여러 기능을 지원합니다.

### 1.2 토크나이저 불러오기

BERT 모델에 사용되는 토크나이저를 불러와보겠습니다.

`from_pretrained()`메서드를 사용한다면, 사전학습된 토크나이저를 불러올 수 있습니다.

In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

토크나이저에 텍스트를 입력하여 결과를 한 번 보도록 하겠습니다.

In [7]:
tokenizer("Deep learning has revolutionized the field of natural language processing.")

{'input_ids': [101, 7786, 3776, 1144, 8011, 2200, 1103, 1768, 1104, 2379, 1846, 6165, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

이뿐만 아니라, 특정 모델을 불러올 경우 그에 맞는 토크나이저를 자동으로 불러오는 기능도 AutoTokenizer을 통해 구현할 수 있습니다.

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

위의 BERT 토크나이저와 결과를 비교해볼까요?

### [TODO] 토크나이저에 문장을 넣고 위의 결과와 비교해보세요.

In [9]:
tokenizer("Deep learning has revolutionized the field of natural language processing.")


{'input_ids': [101, 7786, 3776, 1144, 8011, 2200, 1103, 1768, 1104, 2379, 1846, 6165, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

>예시코드
```
tokenizer("Deep learning has revolutionized the field of natural language processing.")
```

두 토크나이저가 모두 같은 결과를 반환하고 있습니다.  

BERT의 토크나이저는 딕셔너리 형태로 세 가지 항목을 반환합니다.

- `input_ids`는 문장의 각 토큰에 해당하는 인덱스입니다.
- `attention_mask`는 토큰이 Attention을 받아야 하는지 여부를 나타냅니다.
- `token_type_ids`는 두 개 이상의 시퀀스가 있을 때 토큰이 어떤 시퀀스에 속하는지를 식별합니다.



토크나이저는 배치 단위로 문장을 받을 수도 있습니다.

In [10]:
batch_sentences = [
    "To be, or not to be: that is the question.",
    "There is nothing either good or bad, but thinking makes it so.",
    "This above all: to thine own self be true.",
]

encoded_inputs = tokenizer(batch_sentences, padding=True)
encoded_inputs

{'input_ids': [[101, 1706, 1129, 117, 1137, 1136, 1106, 1129, 131, 1115, 1110, 1103, 2304, 119, 102, 0], [101, 1247, 1110, 1720, 1719, 1363, 1137, 2213, 117, 1133, 2422, 2228, 1122, 1177, 119, 102], [101, 1188, 1807, 1155, 131, 1106, 4240, 1162, 1319, 2191, 1129, 2276, 119, 102, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]}

토크나이저를 저장하는 방법은 아래와 같이 간단하게 진행할 수 있습니다.  

단 주의할 점은, 아래 경로에 파일의 이름을 적는 것이 아니라, 경로만 적어야 한다는 것입니다. 그 결과로 생성된 경로 안에
```
('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

 ```
 와 같은 파일이 생성된 것을 확인할 수 있습니다.

In [11]:
tokenizer.save_pretrained('./tokenizer')

('./tokenizer\\tokenizer_config.json',
 './tokenizer\\special_tokens_map.json',
 './tokenizer\\vocab.txt',
 './tokenizer\\added_tokens.json',
 './tokenizer\\tokenizer.json')

### 1.3 인코딩과 디코딩
토크나이저를 이용하여 위와 다른 방법으로 인코딩을 진행해보겠습니다.  

우선 토크나이저의 `tokenize()`메서드를 통해 텍스트를 분할할 수 있습니다.

`add_special_tokens=True`로 입력할 경우, 모델에 맞는 특별한 토큰이 추가됩니다.

BERT 모델의 경우 [CLS] 토큰을 통해 분류 작업을, [SEP] 토큰을 활용하여 복수의 문장을 구분하는 작업을 주 태스크와 동시에 진행합니다. 그러므로 BERT 토크나이저를 사용할 때, 해당 모델이 요구하는 특수 토큰을 함께 추가해주는 것이죠.  

만약 BERT가 아닌 다른 모델이라면, 그에 맞는 특별 토큰이 존재할 경우 자동적으로 추가해줍니다.

In [12]:
sentence = "To be, or not to be: that is the question."
tokens = tokenizer.tokenize(sentence,add_special_tokens=True)
tokens

['[CLS]',
 'To',
 'be',
 ',',
 'or',
 'not',
 'to',
 'be',
 ':',
 'that',
 'is',
 'the',
 'question',
 '.',
 '[SEP]']

`convert_tokens_to_ids()`메서드는 분리된 단어들을 인덱스에 매칭시켜줍니다.

In [13]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[101,
 1706,
 1129,
 117,
 1137,
 1136,
 1106,
 1129,
 131,
 1115,
 1110,
 1103,
 2304,
 119,
 102]

만일 모델 학습이 종료되었다면, 출력 결과는 숫자 형태입니다. `decode`메서드를 통해 토큰을 원본 단어로 복원할 수 있습니다.

In [14]:
decoded = tokenizer.decode(ids)
decoded

'[CLS] To be, or not to be : that is the question. [SEP]'

## 2. 텍스트 전처리

### 2.1 패딩

문장의 길이가 동일하지 않을 경우, 언어 모델의 입력으로 사용할 수 없습니다.  

이러한 경우를 방지하기 위하여 패딩을 토크나이저에 적용할 수 있습니다.

아래와 같이 `padding=True`인자를 넣어주게 되면 자동적으로 배치 내 가장 긴 문장을 기준으로 패딩이 적용됩니다.

### [TODO] 예시 문장에 패딩을 첨가하여 토큰화해봅시다.

In [15]:
batch_sentences = [
    "To be, or not to be: that is the question.",
    "There is nothing either good or bad, but thinking makes it so.",
    "This above all: to thine own self be true.",
]
encoded_inputs = tokenizer(batch_sentences, padding=True)
print(encoded_inputs)

{'input_ids': [[101, 1706, 1129, 117, 1137, 1136, 1106, 1129, 131, 1115, 1110, 1103, 2304, 119, 102, 0], [101, 1247, 1110, 1720, 1719, 1363, 1137, 2213, 117, 1133, 2422, 2228, 1122, 1177, 119, 102], [101, 1188, 1807, 1155, 131, 1106, 4240, 1162, 1319, 2191, 1129, 2276, 119, 102, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]}


> 예시코드
```
batch_sentences = [
    "To be, or not to be: that is the question.",
    "There is nothing either good or bad, but thinking makes it so.",
    "This above all: to thine own self be true.",
]
encoded_inputs = tokenizer(batch_sentences, padding=True)
print(encoded_inputs)
```

### 2.2 길이 제한(Truncation)
짧은 문장은 패딩을 통해 긴 문장과 길이를 맞춰줄 수 있지만, 문장의 길이가 너무 길 경우 데이터에 패딩이 지나치게 많이 포함되어 연산에 영향을 줄 수 있습니다.  

이 경우 `truncation=True`인자를 이용하면, 길이 제한을 통해 길이가 긴 문자열들의 일부분을 절삭할 수 있습니다.

최대 길이는 모델에서 허용한 하이퍼파라미터를 따릅니다.

### [TODO] 예시 문장에 패딩과 길이제한을 첨가하여 토큰화해봅시다.

In [16]:
batch_sentences = [
    "To be, or not to be: that is the question.",
    "There is nothing either good or bad, but thinking makes it so.",
    "This above all: to thine own self be true.",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True)
print(encoded_input)

{'input_ids': [[101, 1706, 1129, 117, 1137, 1136, 1106, 1129, 131, 1115, 1110, 1103, 2304, 119, 102, 0], [101, 1247, 1110, 1720, 1719, 1363, 1137, 2213, 117, 1133, 2422, 2228, 1122, 1177, 119, 102], [101, 1188, 1807, 1155, 131, 1106, 4240, 1162, 1319, 2191, 1129, 2276, 119, 102, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]}


> 예시코드
```
batch_sentences = [
    "To be, or not to be: that is the question.",
    "There is nothing either good or bad, but thinking makes it so.",
    "This above all: to thine own self be true.",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True)
print(encoded_input)
```

### 2.3 텐서 변환

`transformers`의 모델 중 일부분은 Pytorch 환경에서 구현되었고, 또 다른 일부는 Tensorflow에서 만들어졌습니다.

이 두 환경은 서로 텐서의 형태를 공유하지 않으므로, 토큰화된 문장을 Pytorch(`pt`) 텐서나 Tensorflow(`tf`) 텐서로 변환할 수 있습니다.

`return_tensors` 인자를 다음과 같이 조작할 수 있습니다.

### [TODO] 예시 문장에 패딩, 길이제한과 텐서 변환을 첨가하여 토큰화해봅시다.

In [17]:
batch_sentences = [
    "To be, or not to be: that is the question.",
    "There is nothing either good or bad, but thinking makes it so.",
    "This above all: to thine own self be true.",
]

encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_input)

{'input_ids': tensor([[ 101, 1706, 1129,  117, 1137, 1136, 1106, 1129,  131, 1115, 1110, 1103,
         2304,  119,  102,    0],
        [ 101, 1247, 1110, 1720, 1719, 1363, 1137, 2213,  117, 1133, 2422, 2228,
         1122, 1177,  119,  102],
        [ 101, 1188, 1807, 1155,  131, 1106, 4240, 1162, 1319, 2191, 1129, 2276,
          119,  102,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}


> 예시코드
```
batch_sentences = [
    "To be, or not to be: that is the question.",
    "There is nothing either good or bad, but thinking makes it so.",
    "This above all: to thine own self be true.",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_input)
```

## 3. 모델

모델을 불러오는 과정도 토크나이저를 사용하는 방법과 크게 다르지 않습니다.  

우선 이미 사전학습된 BERT 모델을 불러와보겠습니다.


In [18]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

c:\Users\Harmony08\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Harmony08\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

### 3.1 커스텀 모델 생성

그러나 경우에 따라 사전학습되지 않은 초기화된 모델을 사용하여 처음부터 훈련시켜야 할 수도 있습니다.

`transformers`에 있는 모델은 `configuration`객체를 통해 초기화할 수 있습니다.

In [19]:
from transformers import BertConfig, BertModel

config = BertConfig()
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



config는 다음과 같은 딕셔너리 형태로 구성되어 있습니다.

In [20]:
model = BertModel(config)

아래 모델의 인자로 `config`를 입력할 경우 모델이 초기화됩니다.

### 3.2 모델 저장

모델 저장기능은 토크나이저 저장과 동일합니다. 인자 내에 문자열로 디렉토리를 입력하면 해당 위치에 모델 파일이 생성됩니다.

In [21]:
model.save_pretrained("bert-base-uncased")

이렇게 간단하게 `transformers`라이브러리 사용법을 익혀보았습니다.  

이제 이를 바탕으로 뉴스 요약 프로젝트를 진행해보도록 하겠습니다.

---

# T5 모델을 이용한 뉴스 요약 프로젝트

요약 생성 메커니즘에는 2가지 유형이 있습니다:

추출적 요약 (Extractive Summary)

- 정의:  
원문에서 가장 중요하다고 판단되는 문장이나 구절을 직접 추출하여 요약을 만드는 방식입니다.

- 특징:
    - 원문에서 직접적으로 문장을 가져오기 때문에, 요약의 문장들은 원문에 모두 존재합니다.

    - 원문의 문맥과 구조를 그대로 유지하기 때문에, 원문의 의미 전달에는 효과적일 수 있습니다.

    - 그러나, 요약의 길이나 구조를 조절하기 어려울 수 있습니다.

추상적 요약 (Abstractive Summary)

- 정의:  
원문의 내용을 이해하고, 그 의미를 기반으로 새로운 문장을 생성하여 요약을 만드는 방식입니다.

- 특징:

    - 원문에 없는 새로운 문장이나 표현을 사용하여 요약을 생성할 수 있습니다.

    - 원문의 주요 내용을 더 간결하고 자연스럽게 전달할 수 있습니다.

    - 딥러닝 기반의 모델, 특히 시퀀스 투 시퀀스(Seq2Seq) 모델을 활용하여 추상적 요약을 구현하는 경우가 많습니다.

    - 원문의 의미를 왜곡할 위험이 있을 수 있으므로, 정확한 요약을 위한 학습이 중요합니다.

이번 실습에서는 **문장 생성**을 통한 **추상적 요약** 태스크를 진행해보겠습니다.


## 1. 모듈 불러오기 및 환경 설정

### 1.1 모듈 불러오기

딥러닝 모델을 구현하고 학습하기 위한 필수 라이브러리들을 불러오겠습니다.

In [22]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler # 데이터 관련 유틸리티

from transformers import T5Tokenizer, T5ForConditionalGeneration # T5 토크나이저와 조건부 생성 모델

이 코드를 통해 필요한 모든 라이브러리와 모듈을 불러왔으므로, 이제 딥러닝 모델의 구성 및 학습을 시작할 준비가 되었습니다.

이어서 딥러닝 모델 학습 시 사용할 하드웨어를 설정하도록 하겠습니다.

### 1.2 하드웨어 설정

In [23]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

딥러닝 모델의 연산이 GPU에서 수행될 수 있게 되면 학습 속도가 크게 향상됩니다. 만약 GPU가 없거나 사용할 수 없는 경우, CPU에서 연산이 수행됩니다.  

### 1.3 하이퍼 파라미터 설정

이번엔 데이터 전처리와 모델 구성에 앞서 하이퍼 파라미터들을 설정하겠습니다.

In [24]:
TRAIN_BATCH_SIZE = 2 # 학습 데이터 배치 크기
VALID_BATCH_SIZE = 2 # 검증 데이터 배치 크기
TRAIN_EPOCHS = 3 # 학습 횟수
VAL_EPOCHS = 1 # 검증 횟수
LEARNING_RATE = 1e-4

MAX_LEN = 512 # 입력 텍스트의 최대 길이
SUMMARY_LEN = 128 # 요약 텍스트의 최대 길이

- `TRAIN_BATCH_SIZE`와 `VALID_BATCH_SIZE`: 학습 및 검증 데이터의 배치 크기를 설정합니다. 배치 크기는 한 번에 처리되는 데이터의 양을 의미하며, GPU 메모리 용량에 따라 조절될 수 있습니다.

- `TRAIN_EPOCHS`와 `VAL_EPOCHS`: 전체 데이터셋에 대해 학습 및 검증을 수행할 횟수를 설정합니다. 에포크가 많을수록 모델은 데이터를 더 많이 볼 수 있지만, 과적합의 위험이 있습니다.

- `LEARNING_RATE`: 모델의 가중치를 업데이트할 때 사용되는 학습률을 설정합니다. 너무 큰 학습률은 학습이 불안정해질 수 있고, 너무 작은 학습률은 학습 속도가 느려질 수 있습니다.

- `MAX_LEN`와 `SUMMARY_LEN`: 입력 텍스트와 요약 텍스트의 최대 길이를 설정합니다. 텍스트 데이터를 처리할 때, 너무 긴 텍스트는 잘라내거나, 너무 짧은 텍스트는 패딩을 추가하여 이 길이에 맞춰 처리합니다.

### 1.4 시드 설정

In [25]:
SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)

이 코드는 딥러닝 실험의 재현성을 보장하기 위해 난수 생성에 사용되는 시드 값을 설정하는 부분입니다.

- `SEED = 42`: 난수 생성을 위한 시드 값을 42로 설정합니다. 이 값은 실험의 재현성을 보장하기 위해 사용되며, 동일한 시드 값으로 여러 번 실험을 실행하면 동일한 결과를 얻을 수 있습니다.

- `torch.manual_seed(SEED)`: 파이토치에서 난수를 생성할 때 사용되는 시드 값을 설정합니다. 이를 통해 파이토치 내부에서 발생하는 모든 무작위 연산의 결과가 동일하게 유지됩니다.

- `np.random.seed(SEED)`: 넘파이에서 난수를 생성할 때 사용되는 시드 값을 설정합니다. 이를 통해 넘파이 연산에서 발생하는 무작위성이 일정하게 유지됩니다.

이러한 설정은 모델 초기화, 데이터 셔플링, 드롭아웃 등의 무작위 연산에서 동일한 결과를 얻기 위해 중요합니다. 실험의 재현성은 연구 결과의 신뢰성을 높이는 데 중요한 요소입니다.

## 2. 데이터 불러오기 및 전처리

이번 시간에는 텍스트 요약(Text summarization)을 위해 뉴스 데이터를 이용하겠습니다.   

데이터셋의 출처는 [캐글](https://www.kaggle.com/datasets/sunnysai12345/news-summary?select=news_summary_more.csv)로, Inshorts에서 요약된 뉴스로 구성되어 있습니다. 스크랩된 기사의 출처는 Hindu, Indian times, Guardian이며 수집 기간은 2017년 2월부터 8월까지입니다.  

총 4514행의 샘플로 구성되어 있으며, 각 샘플에 대하여 6가지 정보가 열 단위로 저장되어 있습니다.  

- author : 기사의 저자
- date : 기사가 발행된 날짜
- headline : 발행된 기사의 헤드라인
- read_more : 온라인으로 기사를 따라가기 위한 URL
- text : 기사의 요약문
- ctext : 전체 기사

우리는 `ctext`의 내용을 요약하여 `text`와 같은 형태로 출력하는 모델을 만들어 보겠습니다.

### 2.1 데이터 불러오기

아래 경로에서 데이터를 불러옵니다.

In [27]:
data_path = 'content/news_summary.csv'

데이터를 Pandas dataframe으로 저장합니다. 일부 문자가 `latin-1` 인코딩으로 저장되어 있으므로 해당 인자를 입력해줍니다.



In [28]:
raw_news = pd.read_csv(data_path, encoding='latin-1')
raw_news.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


상당히 많은 정보가 존재하지만, 우리에겐 기사의 원문과 요약된 텍스트 두 가지만 필요합니다.  

해당 정보만 따로 추출하여 `df` 데이터프레임에 할당해봅시다.



### [TODO] 데이터프레임에서 요약된`text`와 `ctext`열만 선택하여 슬라이싱합니다.

In [29]:
df = raw_news[['text','ctext']]

In [30]:
df

,text,ctext
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
...,...,...
4509,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m..."
4510,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...
4511,"Aamir Khan, while talking about reality shows ...","Aamir Khan, whose last film Dangal told the st..."
4512,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...


> 예시코드
```
# 'text'와 'ctext' 열만 선택하여 새로운 데이터프레임 생성
df = raw_news[['text','ctext']]
```

### 2.2 요약용 텍스트 태깅

텍스트를 분할했다면, 요약할 대상 문장의 앞에 `'summarize: '`태그를 달아주어야 합니다.

이번 실습에 사용할 `T5(Text-to-Text Transfer Transformer)` 모델은 텍스트를 입력받아 텍스트를 출력하는 구조로 설계되었습니다. 이 모델의 특징 중 하나는 다양한 자연어 처리 작업을 "텍스트 변환" 문제로 간주하고, 특정 작업을 수행하기 위한 명령어를 입력 텍스트의 일부로 제공하는 것입니다.  

예를 들어, 문장 분류 작업을 수행하려면 `"classify: {문장}"`과 같은 형식으로 입력을 제공하고, 번역 작업을 수행하려면 `"translate English to French: {문장}"`과 같은 형식으로 입력을 제공합니다.  

이러한 방식을 사용하는 이유는 T5 모델 뿐만 아니라 대부분의 대형 언어모델을 하나의 일관된 구조로 다양한 작업에 적용할 수 있게 하기 위함입니다. 모델은 입력 텍스트에 포함된 명령어를 통해 어떤 작업을 수행해야 하는지 판단하게 됩니다.  

따라서 `"summarize: "` 문자열을 추가하는 것은 T5 모델에게 텍스트 요약 작업을 수행하도록 지시하는 것과 같습니다. 이와 같은 형식으로 입력을 제공하면, 모델은 주어진 원문을 요약한 결과를 출력하게 됩니다.

### [TODO] 데이터프레임의 `ctext`열 중 모든 데이터에 대해 태그를 달고. 데이터프레임의 `raw_news` 컬럼에 내용을 추가해주세요.

예로, 텍스트가 `"To be, or not to be: that is the question."`일 경우, 이를

`"summarize: To be, or not to be: that is the question."`형태로 바뀌어야 합니다.

In [31]:
df['raw_news'] = 'summarize: ' + df['ctext']
df.head()

C:\Users\Harmony08\AppData\Local\Temp\ipykernel_10180\1670746299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['raw_news'] = 'summarize: ' + df['ctext']


,text,ctext,raw_news
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...,summarize: The Daman and Diu administration on...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo...",summarize: From her special numbers to TV?appe...
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...,summarize: The Indira Gandhi Institute of Medi...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,summarize: Lashkar-e-Taiba's Kashmir commander...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...,summarize: Hotels in Mumbai and other Indian c...


>예시코드
```
# 'ctext' 열 앞에 'summarize: ' 문자열 추가
df['raw_news'] = 'summarize: ' + df['ctext']

# 데이터프레임의 처음 5줄 출력
print(df.head())
```

### 2.3 데이터 분할

`df` 데이터프레임에서 데이터를 나누어 학습용 데이터와 테스트용 데이터로 분할해봅시다.  

전체 데이터의 80%를 학습용으로 사용하도록 하겠습니다. 그러나 학습의 균일함을 위해 데이터프레임의 행들을 섞어 무작위로 분배하도록 하겠습니다.

In [32]:
train_size = 0.8

train_data = df.sample(frac=train_size, random_state=SEED).reset_index(drop=True)

`df.sample(frac=train_size, random_state=SEED)`: `sample` 함수는 데이터프레임에서 무작위로 행을 선택하는 함수입니다. `frac` 인자는 선택할 행의 비율을 나타내며, 여기서는 `train_size`로 설정하여 80%의 행을 선택하도록 합니다. `random_state` 인자는 무작위 선택의 재현성을 보장하기 위해 사용되며, 이전에 설정한 SEED 값을 사용합니다.

`.reset_index(drop=True)`: 선택된 행의 인덱스를 재설정합니다. `drop=True`는 기존 인덱스를 새로운 열로 추가하지 않고 삭제하도록 합니다.

완료가 되었다면 일부분을 보도록 하겠습니다.

In [33]:
train_data.head()

,text,ctext,raw_news
0,"All restaurants, including five-star hotels, i...",Come April and you won?t have to go looking fo...,summarize: Come April and you won?t have to go...
1,The Chinese government has banned singer Justi...,"?I just need one more shot, second chances,? J...","summarize: ?I just need one more shot, second ..."
2,Pakistan on Saturday accused India of targetin...,Pakistan?s top military officer on Saturday ac...,summarize: Pakistan?s top military officer on ...
3,A 40-year-old woman in Hyderabad was burnt ali...,"In a tragic incident, a woman was burnt alive ...","summarize: In a tragic incident, a woman was b..."
4,The Delhi Police has arrested a man working as...,A man who allegedly threatened to blow up Red ...,summarize: A man who allegedly threatened to b...


데이터가 잘 섞여 분배되었습니다. 마찬가지 방법으로 테스트용 데이터셋도 나눠보겠습니다.

### [TODO] `val_data` 변수에 나머지 데이터를 할당해주세요.

In [34]:
val_data = df.drop(train_data.index).reset_index(drop=True)

>예시코드
```
# 학습 데이터셋에 포함되지 않은 나머지 데이터를 검증 데이터셋으로 설정
val_data = df.drop(train_data.index).reset_index(drop=True)
```

`df.drop(train_dataset.index)`: `drop` 함수는 데이터프레임에서 특정 행을 제거하는 함수입니다. 여기서는 `train_dataset.index`를 사용하여 학습 데이터셋에 이미 포함된 행들을 전체 데이터셋(`df`)에서 제거합니다.

`.reset_index(drop=True)`: 제거된 행의 인덱스를 재설정합니다. `drop=True`는 기존 인덱스를 새로운 열로 추가하지 않고 삭제하도록 합니다.

In [35]:
val_data.head()

,text,ctext,raw_news
0,PM Narendra Modi on Thursday launched Ude Desh...,"After much wait, the first UDAN flight took of...","summarize: After much wait, the first UDAN fli..."
1,BJP chief Amit Shah on Wednesday slammed Delhi...,On a day when the Bharatiya Janata Party swept...,summarize: On a day when the Bharatiya Janata ...
2,"Sujal Kahar, a 12-year-old football enthusiast...",NaN,NaN
3,Eight member nations voted against BCCI's prop...,In a massive blow to its influence in the Inte...,summarize: In a massive blow to its influence ...
4,Former AAP leader Mayank Gandhi urged Delhi CM...,Former Aam Aadmi Party leader and a close aide...,summarize: Former Aam Aadmi Party leader and a...


전체 데이터셋과 분할된 데이터셋들의 형태를 확인해볼까요?

In [36]:
print("FULL DataSet:{}".format(df.shape))
print("TRAIN DataSet:{}".format(train_data.shape))
print("VAL DataSet:{}".format(val_data.shape))

FULL DataSet:(4514, 3)
TRAIN DataSet:(3611, 3)
VAL DataSet:(903, 3)


### 2.4 토큰화

이제 준비된 데이터를 바탕으로 토큰화를 진행해보겠습니다. 앞서 설명한 토큰화 방식과 마찬가지로, T5 모델에 대한 토크나이저를 불러오겠습니다.  

In [38]:
tokenzier = T5Tokenizer.from_pretrained('t5-base')

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


 `T5Tokenizer`는 T5 모델의 텍스트를 토큰화하기 위한 토크나이저입니다. `from_pretrained` 메서드를 사용하여 사전 학습된 `"t5-base"` 모델에 대한 토크나이저를 불러옵니다.  

`"t5-base"`는 T5 모델의 기본 버전을 나타냅니다.

### 2.5 데이터셋 클래스 선언

데이터셋 클래스를 선언하도록 하겠습니다.  

아래 데이터셋은 PyTorch의 Dataset 클래스를 상속받아 정의하며, 이를 통해 데이터 로딩 및 전처리를 효율적으로 수행할 수 있습니다.

또한 `DataLoader`와 함께 사용하면 배치 단위로 데이터를 불러와 모델 학습에 사용할 수 있습니다.  

`CustomDataset` 클래스는 T5 모델과 텍스트 요약 작업의 특성을 반영하여 설계되었습니다. 이를 바탕으로, 일반적인 데이터셋과는 다르게 `CustomDataset`이 가져야 하는 특이한 기능들은 다음과 같습니다.

1. 토큰화 및 패딩:

T5 모델은 특정한 토크나이저를 사용하여 텍스트를 토큰화합니다. `CustomDataset` 클래스는 이 토크나이저를 사용하여 원문과 요약문을 토큰화합니다.

또한, 모델에 입력되는 텍스트의 길이는 일정해야 하므로, 주어진 최대 길이에 따라 텍스트를 잘라내거나 패딩을 추가하는 작업이 필요합니다.

2. 반환 형식:

T5 모델 학습을 위해서는 원문(`source_ids`, `source_mask`)과 요약문(`target_ids`)에 해당하는 토큰 ID와 attention mask가 필요합니다.

CustomDataset 클래스는 `__getitem__` 메서드에서 이 정보를 딕셔너리 형태로 반환합니다. 이 딕셔너리는 모델 학습 시 바로 사용될 수 있도록 구성되어 있습니다.

3. 텍스트 정규화:

텍스트 데이터에는 불필요한 공백이나 특수 문자 등이 포함될 수 있습니다. 이 클래스에서는 각 텍스트를 공백을 기준으로 분할한 후 다시 합쳐서 불필요한 공백을 제거하는 작업을 수행합니다.

In [39]:
#사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len, source_col, highlight_col):

        #초기화 매서드
        self.tokenizer = tokenizer
        self.source_len = source_len
        self.summ_len = summ_len
        self.highlight = highlight_col
        self.source = self.data[source_col]

    def __len__(self):
        return len(self.highlight)

    def __getitem__(self, index):
        source_text = str(self.source[index]) #원문 텍스트 추출
        source_text = ''.join(source_text.split()) #텍스트 정규화

        highlight_text = str(self.highlight[index]) #요약문 텍스트 추출
        highlight_text = ''.join(highlight_text.split()) #텍스트 정규화

        #원문 텍스트를 토큰화
        source_encoded = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        #요약문 텍스트를 토큰화
        target_encoded = self.tokenizer.batch_encode_plus(
            [highlight_text],
            max_length=self.summ_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt')

        #토큰화된 결과에서 필요한 정보 추출
        source_ids = source_encoded['input_ids'].sqeeze()
        source_mask = source_encoded['attention_mask'].squeeze()
        target_ids = target_encoded['input_ids'].squeeze()
        target_mask = target_encoded['attention_mask'].squeeze()

        #결과 반환

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }


위 클래스를 뜯어 살펴보겠습니다.

1. 초기화 (`__init__`):

    - 입력으로 주어진 데이터프레임, 토크나이저, 원문과 요약문의 최대 길이, 그리고 원문과 요약문의 열 이름을 기반으로 클래스를 초기화합니다.

    - 원문(source)과 요약문(highlight) 데이터를 멤버 변수로 저장합니다.

2. 데이터셋의 길이 반환 (`__len__`):

    - 데이터셋에 포함된 샘플의 총 개수를 반환합니다.

3. 인덱스에 해당하는 데이터 반환 (`__getitem__`):

    - 주어진 인덱스에 해당하는 원문과 요약문을 추출합니다.

    - 추출된 원문과 요약문을 토크나이저를 사용하여 토큰화합니다. 이때, 주어진 최대 길이에 맞게 텍스트를 잘라내거나 패딩을 추가합니다.

    - 토큰화된 결과에서 필요한 정보 (예: input_ids, attention_mask 등)를 추출합니다.

    - 추출된 정보를 딕셔너리 형태로 반환합니다. 이 딕셔너리는 모델 학습 시 입력 데이터로 사용됩니다.

데이터셋 클래스가 선언되었으므로, 이를 바탕으로 학습용 데이터셋을 만들어보겠습니다.

### [TODO] 학습용 데이터셋 인스턴스를 생성해주세요.

>예시코드
```
training_set = CustomDataset(dataframe=train_data,
                        tokenizer=tokenizer,
                        source_len=MAX_LEN,
                        summ_len=SUMMARY_LEN,
                        source_col='text',
                        highlight_col='ctext',
                        )
```

`tokenizer=tokenizer`: T5 모델의 토크나이저를 사용하여 텍스트를 토큰화합니다.

`source_len=MAX_LEN`: 원문의 최대 길이를 `MAX_LEN`으로 설정합니다. 이 길이를 초과하는 원문은 잘리게 되며, 이 길이보다 짧은 원문은 패딩됩니다.

`summ_len=SUMMARY_LEN`: 요약문의 최대 길이를 `SUMMARY_LEN`으로 설정합니다. 이 길이를 초과하는 요약문은 잘리게 되며, 이 길이보다 짧은 요약문은 패딩됩니다.

`source_col='text'`: 원문 데이터가 포함된 열의 이름을 `'text'`로 설정합니다.

`highlight_col='ctext'`: 요약문 데이터가 포함된 열의 이름을 `'ctext'`로 설정합니다.

CustomDataset 클래스의 `__getitem__`메서드를 호출하여 인덱스 1에 해당하는 샘플을 가져옵니다. 이 메서드는 주어진 인덱스에 해당하는 원문과 요약문을 토큰화하고, 필요한 정보를 딕셔너리 형태로 반환합니다.  

한 번 내부를 살펴볼까요?

### [TODO] 데이터셋 클래스의 1번 인덱스에 해당하는 데이터를 불러와주세요.

> 예시코드
```
sample = training_set.__getitem__(1)
sample
```

각 키와 값에 대한 설명은 다음과 같습니다:

- `source_ids`: 원문(`text`)의 토큰화된 결과입니다. 이는 T5 모델에 입력으로 제공될 원문의 토큰 ID들을 나타냅니다. 값 중 0은 패딩 토큰을 나타냅니다.

- `source_mask`: 원문의 attention mask입니다. 이는 모델이 원문의 어떤 부분에 주의를 기울여야 하는지를 나타냅니다. 1은 해당 위치의 토큰이 실제 데이터를 나타내며, 0은 패딩 토큰을 나타냅니다.

- `target_ids`: 요약문(`ctext`)의 토큰화된 결과입니다. 이는 T5 모델의 출력과 비교될 대상입니다. 값 중 0은 패딩 토큰을 나타냅니다.

- `target_ids_y`: 요약문의 토큰화된 결과입니다. 이는 학습 시 정답 라벨로 사용됩니다. `target_ids`와 동일한 값을 가집니다.

CustomDataset 클래스의 `__getitem__`메서드를 호출하면, 데이터셋 전체 길이를 확인할 수 있습니다.

### [TODO] 테스트 데이터셋 인스턴스를 생성해주세요.

> 예시코드
```
val_set = CustomDataset(dataframe=val_data,
                        tokenizer=tokenizer,
                        source_len=MAX_LEN,
                        summ_len=SUMMARY_LEN,
                        source_col='text',
                        highlight_col='ctext',
                        )
```

### 2.6 Dataloader

Pytorch DataLoader는 데이터셋과 샘플러를 입력으로 받아, 데이터셋에서 데이터를 가져와 배치로 묶어주는 반복 가능한 객체(iterable)를 생성합니다.  

이는 미니배치 학습, 데이터 셔플, 병렬 데이터 로딩 등을 쉽게 수행할 수 있게 해줍니다.

DataLoader을 생성하기에 앞서 파라미터를 설정해주겠습니다.  

파라미터에는 배치 크기, 셔플 여부, 사용할 프로세스 수를 기입해줍시다.

`batch_size`: 한 번에 처리할 데이터의 양을 설정합니다. 배치 크기는 GPU 메모리와 관련이 있으며, 너무 크게 설정하면 메모리 오류가 발생할 수 있습니다.

`shuffle`: 학습 데이터의 경우, 각 에폭마다 데이터를 섞어서 모델이 데이터의 순서에 익숙해지지 않게 합니다. 이를 통해 모델의 일반화 성능을 향상시킬 수 있습니다. 반면, 테스트 데이터는 모델의 성능을 평가하는 데만 사용되므로 데이터를 섞을 필요가 없습니다.

`num_workers`: 데이터 로딩 속도를 높이기 위해 여러 프로세스를 사용할 수 있습니다. 여기서는 단순화를 위해 메인 프로세스에서만 데이터를 로드하도록 설정되었습니다.

딕셔너리 언패킹을 통해 위에서 선언한 파라미터 변수를 데이터로더 생성에 적용합니다.

### [TODO] 데이터로더를 생성해주세요.

위에서 선언한 딕셔너리를 이용하여 데이터로더를 생성해주세요.

> 예시코드
```
# 학습 및 검증을 위한 데이터 로더 생성
training_loader = DataLoader(training_set, **train_params)  # 학습 데이터셋을 위한 데이터 로더
val_loader = DataLoader(val_set, **val_params)              # 테스트 데이터셋을 위한 데이터 로더
```

훌륭합니다. 이제 데이터 준비를 마쳤으니, 모델을 불러와 학습시키기만 하면 됩니다.

## 3. 모델 인스턴스 생성 및 학습

이번 실습에 사용할 모델은 [T5](https://arxiv.org/abs/1910.10683)입니다.  

![T5](https://miro.medium.com/max/4006/1*D0J1gNQf8vrrUpKeyD8wPA.png)

T5(Text-to-Text Transfer Transformer)는 Google Research에서 개발된 트랜스포머 기반의 모델로, 모든 자연어 처리 작업을 "텍스트를 텍스트로 변환하는 작업"으로 간주합니다.  

이 독특한 접근 방식은 다양한 NLP 작업을 동일한 모델 아키텍처와 훈련 방법으로 처리할 수 있게 해주고 이는 사용되는 문장 앞에 독특한 태그를 붙이는 방식으로 진행됩니다.

예를 들어 번역 작업에서는 "translate English to Korean: Hello"와 같은 입력을 받아 "안녕하세요"라는 출력을 생성합니다.

Hugging Face에서 배포되는 다른 LLM과 마찬가지로, 큰 텍스트 데이터셋에서 미리 훈련된 후 특정 작업에 미세 조정되어 다양한 크기(small, base, large, 3B, 11B 등)로 제공됩니다.

우리는 이 모델이 제공하는 다양한 기능 중 요약기능("summarize: ")을 통해 위에서 처리한 뉴스 데이터를 요약해보도록 하겠습니다.

우선 Hugging Face Transformers 라이브러리에서 T5 모델을 불러오도록 하겠습니다.

### 3.1 모델 불러오기

- `T5ForConditionalGeneration`: T5 모델의 변형으로, 조건부 생성 작업(예: 텍스트 요약)을 수행하기 위해 설계되었습니다. 이 모델은 주어진 입력 텍스트에 대한 요약을 생성하는 데 사용됩니다.

- `.from_pretrained("t5-base")`: 사전 학습된 't5-base' 모델을 불러옵니다. 't5-base'는 T5 모델의 중간 크기 버전으로, 광범위한 텍스트 데이터에서 사전 학습되었습니다.

- `.to(device)`: 모델을 현재 사용 가능한 하드웨어 장치(GPU 또는 CPU)로 전송합니다.

### 3.2 Optimizer
모델에 사용할 옵티마이저를 불러오고, 이에 적용할 학습률을 설정합니다.

`params=model.parameters()`인자는 모델의 모든 파라미터(가중치 및 편향)를 옵티마이저에 전달하여 학습 중에 업데이트될 수 있도록 합니다.

### 3.3 학습 함수 선언

학습에 필요한 함수를 선언합니다.  

모델을 학습모드로 설정하여 `순전파`, `손실계산`, `역전파`, `가중치 업데이트` 총 네 가지 과정을 for 문을 통해 반복적으로 진행합니다.

- 모델 학습 모드 설정: `model.train()`을 사용하여 모델을 학습 모드로 설정합니다. 이렇게 하면 모델 내의 드롭아웃과 같은 특정 레이어가 학습 중에만 활성화됩니다.

- 데이터 로딩: loader를 통해 학습 데이터를 배치 단위로 가져옵니다.

- 데이터 전처리: 타겟 데이터(`y`)를 GPU로 이동시키고, 입력(`ids, mask`) 및 레이블(`lm_labels`)로 사용될 데이터를 준비합니다. `lm_labels`는 손실 계산에 사용되며, 패딩 토큰 위치에 -100 값을 가집니다.

    - PyTorch의 크로스 엔트로피 손실 함수는 -100 값을 가진 레이블을 자동으로 무시하도록 설계되어 있습니다.

    - 패딩 토큰들은 시퀀스의 길이를 동일하게 맞추기 위해 사용되지만, 실제 데이터가 아니므로 손실 계산에서 무시되어야 합니다.

- 모델 전달: 입력 데이터를 모델에 전달하여 출력을 얻습니다. 이 때, 손실도 함께 반환됩니다.

- 손실 출력: 500번째 스텝마다 현재의 손실을 출력하여 학습 진행 상황을 모니터링합니다.

- 역전파 및 가중치 업데이트: 손실을 기반으로 역전파를 수행하고, 옵티마이저를 사용하여 모델의 가중치를 업데이트합니다.

### 3.4 평가 함수 선언

이번엔 평가에 사용할 함수를 선언하겠습니다.  

학습 함수와 유사하지만, `순전파`와 `손실계산`까지만 진행된다는 점이 다릅니다.

- 모델 평가 모드 설정: `model.eval()`을 사용하여 모델을 평가 모드로 설정합니다. 이렇게 하면 모델 내의 드롭아웃과 같은 특정 레이어가 비활성화됩니다.

- 데이터 로딩: loader를 통해 검증 데이터를 배치 단위로 가져옵니다.

- 데이터 전처리: 타겟 데이터(`y`)와 입력 데이터(`ids, mask`)를 GPU로 이동시킵니다.

- 모델 전달 및 요약 생성: 입력 데이터(`ids, mask`)를 모델에 전달하여 요약을 생성합니다. 이 때, `model.generate` 메서드를 사용하여 주어진 입력에 대한 요약을 생성합니다. 여기에는 빔 서치, 반복 패널티, 길이 패널티 등의 다양한 설정이 포함됩니다.

    - 빔 서치와 두 페널티에 관한 설명은 아래에 자세히 이어서 설명하겠습니다.

- 토큰 ID를 텍스트로 변환: 생성된 요약의 토큰 ID와 실제 요약의 토큰 ID를 실제 텍스트로 변환합니다. 이 때, `tokenizer.decode` 메서드를 사용하여 토큰 ID를 텍스트로 변환합니다.

- 진행 상황 모니터링: 100번째 스텝마다 현재 진행 상황을 출력하여 검증 진행 상황을 모니터링합니다.

- 결과 반환: 모든 배치에 대한 예측이 완료되면 생성된 요약과 실제 요약을 반환합니다. 이를 통해 후에 성능 지표를 계산할 수 있습니다.

### 3.5 Beam Search

빔 서치는 텍스트 생성 작업에서 가장 가능성 있는 시퀀스를 찾기 위한 알고리즘입니다.  

그리디 탐색(Greedy Search)은 각 단계에서 가장 확률이 높은 토큰만을 선택하는 반면, 빔 서치는 여러 후보 시퀀스를 동시에 고려하면서 생성 작업을 진행합니다.

![Beam search](https://velog.velcdn.com/images%2Fdldydldy75%2Fpost%2F714a88b6-a16a-4477-989b-f5a0782090db%2Fimage.png)

빔 서치의 핵심 개념은 '빔 너비(beam width)'입니다. 빔 너비는 알고리즘이 각 단계에서 고려하는 후보 시퀀스의 수를 나타냅니다.  

빔 너비가 1이면 그리디 탐색과 동일하게 작동하며, 빔 너비가 높아질수록 더 많은 후보 시퀀스를 고려하게 됩니다. 그러나 빔 너비가 너무 크면 계산 복잡도가 증가하게 됩니다.

빔 서치는 여러 후보 시퀀스 중에서 최적의 시퀀스를 선택하는 데 유용하지만, 몇 가지 문제점도 있습니다.

이러한 문제를 해결하기 위해 반복 페널티와 길이 페널티가 도입되었습니다.

- 반복 페널티 (Repetition Penalty):

    - 텍스트 생성 중에 동일한 단어나 구절이 반복적으로 나타나는 것을 방지하기 위한 페널티입니다.

    - 반복 페널티 값이 1보다 큰 경우, 모델이 이전에 생성한 토큰을 다시 생성하는 것에 페널티를 부여합니다. 이로 인해 생성된 텍스트에서 반복적인 내용이 줄어듭니다.

    - 반복 페널티 값을 조절함으로써 생성 텍스트의 반복성을 조절할 수 있습니다.

- 길이 페널티 (Length Penalty):

    - 빔 서치에서는 여러 후보 시퀀스 중에서 가장 확률이 높은 시퀀스를 선택합니다.
    
    - 그러나 짧은 시퀀스는 자연스럽게 높은 확률을 가질 수 있기 때문에, 길이 페널티를 도입하여 긴 시퀀스의 확률이 과도하게 감소되는 것을 방지합니다.

    - 길이 페널티 값이 1인 경우, 페널티가 적용되지 않습니다.
    
    - 값이 1보다 크면 긴 시퀀스에 대한 확률이 상대적으로 증가하게 됩니다.

    - 이 페널티는 생성된 텍스트의 길이를 조절하는 데 도움을 줍니다.

### 3.6 학습

두 함수가 완성되었으므로, 반복문과 이들을 이용하여 학습을 진행하도록 하겠습니다.

매 에포크마다 학습과 검증을 수행하고, 모델의 예측값과 정답을 데이터프레임이 저장하도록 하겠습니다.  

학습을 완료하는 데에는 T4 GPU(VRAM 16G) 기준 1시간 20분 가량 소요됩니다. 그 동안 커피 한 잔의 여유를 즐기며 앞서 배웠던 내용을 살펴볼까요?  

T5 모델에 대해 더 알아보셔도 좋고, Beam search에 대한 내용을 다시 한 번 확인해보셔도 좋습니다.

### [TODO] 위에서 선언한 학습 함수와 검증 함수를 이용하여 모델을 훈련시킵니다.

`for`루프를 통해 매 에포크마다 학습과정과 검증과정을 수행하도록 코드를 완성시켜주세요.

> 예시코드
```
# TRAIN_EPOCHS 동안 모델을 학습시키는 루프
for epoch in range(TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, optimizer)  # 각 에포크마다 train 함수를 호출하여 모델을 학습

# VAL_EPOCHS 동안 모델을 검증하고 결과를 predictions.csv 파일로 저장하는 루프
for epoch in range(VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)    # 각 에포크마다 validate 함수를 호출하여 검증 데이터셋에 대한 모델의 성능을 평가
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})   # 생성된 예측과 실제 값을 포함하는 데이터프레임(final_df)을 생성
    final_df.to_csv('predictions.csv')                                              # predictions.csv 파일로 저장
    print('Output Files generated for review')                                      # 파일이 생성되었음을 알리는 메시지를 출력
```

학습이 완료되었다면, 모델을 저장합니다.

## 4. Rouge 스코어를 통한 평가

학습이 완료되었다면, 결과물을 불러와봅시다.  

`predictions.csv`를 불러와 데이터프레임으로 저장해주세요.

### 4.1 결과 비교

반복문을 통해 데이터프레임의 상위 15개 샘플에 대한 예측값과 실제 결과를 비교해봅시다.

다소 차이가 있지만, 비슷한 단어들이 예측 결과와 레이블에서 동시에 발견됩니다.  

특정 문장의 경우 레이블이 없음에도 잘 생성된 것을 볼 수 있습니다.

다만 아쉬운 점은, 이렇게 육안으로 비교할 경우 정량적인 결과 분석이 어렵습니다. 이러한 경우를 위해 Rouge 스코어로 평가를 해보겠습니다.

### 4.2 Rouge score

Rouge(R-ecall O-riented U-nderstudy G for E-valuation)는 자동 요약의 성능을 평가하기 위한 메트릭 중 하나입니다. Rouge는 여러 버전이 있으며, 각 버전은 다른 방식으로 요약의 품질을 평가합니다.

- ROUGE-N: N-gram 기반의 정밀도와 재현율을 계산합니다. 예를 들어,
ROUGE-1은 unigram, ROUGE-2는 bigram에 기반한 스코어를 제공합니다.

- ROUGE-L: 가장 긴 공통 부분 문자열(Longest Common Subsequence, LCS)을 기반으로 합니다. LCS는 두 문자열 사이에서 순서를 변경하지 않고 얻을 수 있는 가장 긴 공통 부분 문자열을 찾는 것을 의미합니다.

- ROUGE-S: skip-bigram을 기반으로 합니다. Skip-bigram은 문장 내에서 몇 개의 단어를 건너뛰더라도 순서가 유지되는 단어 쌍을 의미합니다.

Rouge 스코어는 주로 정밀도(Precision), 재현율(Recall), F1 스코어로 제공됩니다.

- 정밀도는 생성된 요약 내에서 실제 요약과 일치하는 단어나 구의 비율을 나타내며,

- 재현율은 실제 요약 내에서 생성된 요약과 일치하는 단어나 구의 비율을 나타냅니다.

- F1 스코어는 정밀도와 재현율의 조화 평균입니다.

### 4.3 결측 문장 확인 및 제거

해당 평가지표를 적용하기 위해 결측치를 확인해보겠습니다.

29개의 샘플에 대해 레이블 데이터가 없다고 확인되네요.  

해당 샘플을 제거해줍시다.

### 4.4 Rouge 스코어 측정

결과 데이터프레임에 대해 Rouge 스코어를 측정해보도록 하겠습니다.

각 열의 값을 리스트로 변환 후 스코어를 계산하고, 결과를 출력해보도록 하겠습니다.

### [TODO] Rouge 스코어를 계산해주세요.

아래 코드의 빈 칸을 채워넣어 실제 문장과 예측 문장 사이의 점수를 측정하도록 하겠습니다.

> 예시코드
```
from rouge import Rouge  # Rouge 평가 도구를 가져옴

# 'Generated Text' 열의 값을 문자열 리스트로 변환하여 predictions에 저장
predictions = [str(text) for text in pred['Generated Text'].tolist()]
# 'Actual Text' 열의 값을 문자열 리스트로 변환하여 actuals에 저장
actuals = [str(text) for text in pred['Actual Text'].tolist()]

rouge = Rouge()  # Rouge 객체 생성
# 생성된 요약과 실제 요약 사이의 Rouge 스코어 계산
scores = rouge.get_scores(predictions, actuals, avg=True)

print(scores)  # Rouge 스코어 출력
```

각 지표는 다음과 같은 의미를 가집니다:

- rouge-1: unigram (단일 단어)의 일치도를 나타냅니다.

- rouge-2: bigram (두 단어의 조합)의 일치도를 나타냅니다.

- rouge-l: 가장 긴 공통 부분 문자열의 일치도를 나타냅니다.
각 지표에는 세 가지 값이 있습니다:

- r (recall): 실제 요약에서 얼마나 많은 단어/구문이 생성된 요약에 포함되었는지 나타냅니다.

- p (precision): 생성된 요약에서 실제 요약과 일치하는 단어/구문의 비율을 나타냅니다.

- f (f-score): recall과 precision의 조화 평균입니다. 두 지표의 균형을 나타내는 값으로, 높을수록 좋습니다.

다만, 지표가 딕셔너리로 제공되기에 가시성이 떨어집니다.  

아래 코드를 통해 딕셔너리의 키와 값을 뽑아내어 정제된 형태로 출력해보겠습니다.  

이렇게 T5 모델을 통해 결과를 측정해보았습니다. 요약에는 T5 이외에도 [Bart](https://huggingface.co/facebook/bart-base), [Pegasus](https://huggingface.co/google/pegasus-large) 등이 자주 사용됩니다.  이러한 모델을 사용하여 요약을 진행해보고, 결과를 비교해보는 것은 어떨까요?

또, [Hugging Face Datasets](https://huggingface.co/datasets?task_categories=task_categories:summarization&sort=trending)에는 수많은 요약용 데이터가 준비되어있습니다. 이를 이용하여 여러분들의 실력을 키워보시는 것도 추천드립니다.

이번 실습도 수고많으셨습니다!